# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのエージェントを開発し、実装することに焦点を当てています。特に、質問応答ゲームである「20の質問」において、AI言語モデルを活用して効率的な質問と回答を生成し、ターゲットワードを推測するタスクに取り組みます。

### 主な取り組み内容

1. **ライブラリとモデルのセットアップ**:
   - 必要なPythonパッケージ（`rigging`、`kaggle`、`transformers`、`accelerate`、`bitsandbytes`）をインストールし、KaggleのシークレットからHugging FaceトークンやKaggleのユーザー情報を取得しています。
   - Hugging Face HubからOLM（Optimized Language Model）のモデルとして「Meta-Llama-3-8B-Instruct-bnb-8bit」をダウンロードし、動作確認を行います。

2. **モデルの実装**:
   - 主要なエージェント関数 `agent_fn` では、観察結果に基づいて質問、回答、推測を行うロジックが実装されています。
   - 質問は情報を集めるように限られた回答の選択肢（「はい」、「いいえ」、「もしかしたら」）を考慮して生成され、観察内容をXML形式で追跡します。
   - 各エージェントは、現在のターンに従って適切なアクションを取ります。

3. **ファイルの圧縮と提出**:
   - 完成したモデルやライブラリを含むアーカイブを作成し、Kaggleに提出するための準備をしています。`pigz`および`pv`といったツールを使用し、効率的な圧縮処理を行っています。

### 使用ライブラリ
- **Kaggle API**: コンペティションへのデータ提出をサポート。
- **Hugging Face Hub**: モデルの取得と管理。
- **Rigging**: モデル生成に関連する高度な操作を提供。

### 結論
このNotebookは、AIエージェントによる「20の質問」ゲームの最適化に重点を置いたものであり、戦略的かつ効率的な質問応答を通じてターゲットワードを素早く特定することが求められています。データの取り扱いやモデルの実装に専門的な手法とライブラリが利用されていることが特徴です。

---


# 用語概説 
以下に、与えられたJupyter Notebookの内容に基づいて、機械学習や深層学習の初心者がつまずきそうな専門用語の簡単な解説を列挙します。特にあまり馴染みのない用語や、このノートブック特有のドメイン知識に焦点を当てています。

1. **rigging**:
   - 特殊なライブラリで、特に大規模な言語モデルを操作するための機能を提供します。具体的には、モデルの生成やチャットインターフェースの構築に役立つツール群です。このコンテキストでは、言語モデルの性能を最大限に引き出すために使用されます。

2. **Hugging Face Hub**:
   - Hugging Faceのオンラインプラットフォームで、さまざまな事前学習されたモデルを共有・ダウンロードできるリポジトリです。特に自然言語処理(NLP)の分野で人気のモデルが多く公開されています。

3. **snapshot_download**:
   - Hugging Face Hubから特定のモデルのスナップショット（状態）をダウンロードするための関数。このメカニズムにより、モデルのバージョン管理や安定した環境での再現が可能になります。

4. **CUDA**:
   - NVIDIAの並列計算プラットフォームおよびAPIで、GPU（グラフィックス処理ユニット）を利用して計算処理を高速化する技術。深層学習モデルのトレーニングや推論を効率的に行うために使用されます。

5. **iterators**:
   - Pythonのコレクションやデータ構造に対して反復処理を行うためのオブジェクト。特に、ループ処理を行うための特別なメソッドを持ち、データを1つずつ取り出して操作する機能を提供します。

6. **Pydantic**:
   - Pythonのデータバリデーションと設定管理ライブラリ。型安全性を重視しており、データモデルの検証やシリアライズを簡単に行えるようにします。このノートブックでは、API入力の検証に使用されています。

7. **field_validator**:
   - Pydanticの機能の一部で、特定のフィールドに対してカスタムなバリデーションロジックを提供します。モデルが期待する値に適合しない場合にエラーを投げる役割を持ちます。

8. **Annotated**:
   - Pythonの型ヒント機能で、型に追加情報を付加するために使用されます。この情報は、例えば、データが持つべき制約（空白を削除するなど）の定義に用いられます。

9. **xml_example**:
   - XMLフォーマットの例を生成するメソッド。データをXML形式で表現する際に利用され、他のシステムとのデータのやり取りを容易にするための補助的な役割を果たします。

10. **Contextualized Prompting**:
   - モデルに対して文脈を考慮した質問をする技術。状態に応じて適切な情報を提供することで、モデルの応答を向上させるための手法です。このノートブックでは、特定の入力から有用な出力を得るために重要です。

これらの専門用語は、初心者が理解するのに役立つ基本的な知識を深めるための出発点になるでしょう。

---


In [ ]:
# 依存パッケージのインストール

# 必要なパッケージを指定の場所にインストールします
!pip install -U \
    -t /kaggle/tmp/lib \  # インストール先のディレクトリ
    rigging \  # riggingパッケージをインストール
    kaggle \  # kaggleパッケージをインストール
    transformers \  # transformersライブラリをインストール
    accelerate \  # accelerateライブラリをインストール
    -i https://pypi.org/simple/ bitsandbytes  # bitsandbytesを指定のリポジトリからインストール

In [ ]:
# シークレットの取得

# Kaggleのシークレット機能を利用するために必要なライブラリをインポートします
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()  # UserSecretsClientのインスタンスを作成

# Hugging Faceのトークンを取得します
HF_TOKEN = secrets.get_secret("HF_TOKEN")  # HF_TOKENという名前のシークレットを取得

# Kaggleのキーとユーザー名を取得します
KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")  # KAGGLE_KEYという名前のシークレットを取得
KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")  # KAGGLE_USERNAMEという名前のシークレットを取得

In [ ]:
# モデルのダウンロード

# Hugging Face Hubからモデルをダウンロードするために必要なライブラリをインポートします
from huggingface_hub import snapshot_download  # モデルのスナップショットをダウンロードするための関数
from pathlib import Path  # パス操作のためのライブラリ
import shutil  # ファイル操作のためのライブラリ

# モデル保存用のパスを定義
g_model_path = Path("/kaggle/tmp/model")
# 既にモデルのパスが存在する場合は削除します
if g_model_path.exists():
    shutil.rmtree(g_model_path)  # 既存のモデルディレクトリを削除
# 新しいモデル用のディレクトリを作成します
g_model_path.mkdir(parents=True)

# モデルを指定したリポジトリからダウンロードします
snapshot_download(
    repo_id="alokabhishek/Meta-Llama-3-8B-Instruct-bnb-8bit",  # ダウンロードするモデルのリポジトリID
    local_dir=g_model_path,  # モデルを保存するローカルディレクトリ
    local_dir_use_symlinks=False,  # シンボリックリンクを使用しない設定
    token=HF_TOKEN  # Hugging Faceのトークンを使用して認証
)

In [ ]:
# モデルの読み込み確認

# システムのパスにライブラリのディレクトリを追加します
import sys
sys.path.insert(0, "/kaggle/tmp/lib")  # カスタムライブラリのパスを最初に追加

# riggingライブラリをインポートします
import rigging as rg

# モデルを生成するための関数を呼び出します
generator = rg.get_generator("transformers!/kaggle/tmp/model,device_map=cuda:0")  # CUDAデバイスを使用してモデルを取得

# チャット機能を使用して「Hello!」と発言させます
chat = generator.chat("Say Hello!").run()

# チャットの最後の返答を出力します
print(chat.last)  # チャットの最後のメッセージを表示

In [ ]:
%%writefile main.py

# モデルの実装

import itertools
import os
import sys
import typing as t
from pathlib import Path

# パスの修正

g_working_path = Path('/kaggle/working')  # 作業ディレクトリのパス
g_input_path = Path('/kaggle/input')  # 入力データのパス
g_temp_path = Path("/kaggle/tmp")  # 一時ファイル用のパス
g_agent_path = Path("/kaggle_simulations/agent/")  # エージェント用のパス
g_model_path = g_temp_path / "model"  # モデルのパス

# エージェントのパスが存在する場合の処理
if g_agent_path.exists():
    sys.path.insert(0, str(g_agent_path / "lib"))  # エージェントのライブラリをパスに追加
    g_model_path = g_agent_path / "model"  # モデルのパスを更新
else:
    sys.path.insert(0, str(g_temp_path / "lib"))  # 一時ファイルのライブラリをパスに追加

import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_generator_id = f"transformers!{g_model_path},trust_remote_code=True,max_tokens=1024,temperature=1.0,top_k=256"  # モデル生成のためのID設定

# 型

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]  # 空白を削除するための型定義

class Observation(BaseModel):
    step: int  # 現在のステップ
    role: t.Literal["guesser", "answerer"]  # 役割（予測者または回答者）
    turnType: t.Literal["ask", "answer", "guess"]  # ターンタイプ（質問、回答、推測）
    keyword: str  # キーワード
    category: str  # カテゴリー
    questions: list[str]  # 質問のリスト
    answers: list[str]  # 回答のリスト
    guesses: list[str]  # 推測のリスト
    
    @property
    def empty(self) -> bool:
        # すべての質問、回答、推測が空であればTrueを返す
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        # 歴史（質問、回答、推測）を取得
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        # XML形式で歴史を取得
        return "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
        ) if not self.empty else "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no", "maybe"]  # 回答の内容

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        # 回答が有効な値かどうかを検証
        for valid in ["yes", "no", "maybe"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です。有効な回答のいずれかでなければなりません：'yes', 'no', 'maybe'")

    @classmethod
    def xml_example(cls) -> str:
        # XML形式の例を返す
        return f"{Answer.xml_start_tag()}**yes/no/maybe**{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip  # 質問の内容

    @classmethod
    def xml_example(cls) -> str:
        # XML形式の質問の例を返す
        return Question(content="**question**").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip  # 推測の内容

    @classmethod
    def xml_example(cls) -> str:
        # XML形式の推測の例を返す
        return Guess(content="**thing/place/person**").to_pretty_xml()


# 関数群

def ask(base: rg.PendingChat, observation: Observation) -> str:
    # 次の質問を生成する関数
    chat = (
        base.fork(
            f"""\
            あなたは現在次の質問をしています。

            <game-history>
            {observation.get_history_as_xml(skip_guesses=True)}
            </game-history>

            上記の歴史に基づいて、次に最も有用なはい/いいえの質問をし、以下の形式で記載してください：
            {Question.xml_example()}

            - あなたの回答は、最も情報を集められるような焦点を合わせた質問である必要があります
            - 質問は一般的なもので始めてください
            - 残りの検索空間を二分するように常に努力してください
            - 順前の質問と回答に注意を払ってください

            あなたの次の質問は何ですか？
            """
        )
        .until_parsed_as(Question, attempt_recovery=True)  # 質問がパースされるまで実行
        .run()
    )
    return chat.last.parse(Question).content  # 質問の内容を返す


def answer(base: rg.PendingChat, observation: Observation) -> t.Literal["yes", "no", "maybe"]:
    # 質問に回答する関数
    last_question = observation.questions[-1]  # 最後の質問を取得
    chat = (
        base.fork(
            f"""\
            このゲームの秘密の言葉は「{observation.keyword}」です。

            あなたは現在上記の作業に関する質問に答えています。

            次の質問は「{last_question}」です。

            上記のはい/いいえの質問に答え、以下の形式で記載してください：
            {Answer.xml_example()}

            - あなたの回答は上記のキーワードに対して正確である必要があります
            - 常に「yes」、「no」、「maybe」で答えてください

            あなたの回答は何ですか？
            """
        )
        .until_parsed_as(Answer, attempt_recovery=True)  # 回答がパースされるまで実行
        .run()
    )
    return chat.last.parse(Answer).content  # 回答の内容を返す


def guess(base: rg.PendingChat, observation: Observation) -> str:
    # キーワードの推測を行う関数
    pending = (
        base.fork(
            f"""\
            あなたは現在キーワードの情報に基づいた推測を行っています。

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            上記の歴史に基づいて、キーワードの次の最良の推測を生成し、以下の形式で記載してください：
            {Guess.xml_example()}

            - 上記の歴史に基づいて、同じ推測を避けてください
            - 推測は具体的人、場所、または物でなければなりません

            あなたの推測は何ですか？
            """
        )
        .until_parsed_as(Guess, attempt_recovery=True)  # 推測がパースされるまで実行
        .run()
    )
        
    return chat.last.parse(Guess).content  # 推測の内容を返す

# モデル生成器の取得

generator = rg.get_generator(g_generator_id)

# エントリーポイント

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # エージェントの主要な関数
    observation = Observation(**obs.__dict__)  # 入力されたデータからObservationオブジェクトを作成
    
    try:
        base = generator.chat("""\
            あなたは20の質問ゲームの才能あるプレイヤーです。あなたは正確で、焦点を絞り、
            構造化されたアプローチを取ります。役に立つ質問を作成し、推測を行い、
            キーワードに関する質問に答えます。
            
            """
        )
    
        match observation.turnType:
            case "ask":
                return ask(base, observation)  # 質問を行う
            case "answer":
                if not observation.keyword:
                    return "maybe"  # キーワードがない場合は「maybe」と返す
                return answer(base, observation)  # 回答を行う
            case "guess":
                return guess(base, observation)  # 推測を行う
            case _:
                raise ValueError("未知のターンタイプです")  # 不明なターンタイプのエラーを発生させる
    except Exception as e:
        print(str(e), file=sys.stderr)  # エラーを標準エラー出力に表示
        raise  # エラーを再度発生させる

In [ ]:
# 必要なパッケージのインストール

# pigzとpvをインストールします
!apt install pigz pv  # pigzは並列圧縮ツール、pvはデータの視覚化ツールです

In [ ]:
# submission.tar.gzの作成

# 提出用のtar.gzアーカイブを作成します
!tar --use-compress-program='pigz --fast' \  # pigzを利用して圧縮
    -cf submission.tar.gz \  # アーカイブファイルの名前
    --dereference \  # シンボリックリンクをたどって解凍する
    -C /kaggle/tmp model lib \  # /kaggle/tmp内のmodelとlibディレクトリを追加
    -C /kaggle/working main.py  # /kaggle/working内のmain.pyを追加

In [ ]:
# Kaggleへの直接提出

# Kaggleに競技会の提出を行います
!KAGGLE_USERNAME={KAGGLE_USERNAME} \  # Kaggleのユーザー名を設定
 KAGGLE_KEY={KAGGLE_KEY} \  # KaggleのAPIキーを設定
 kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "Updates"  # 指定されたコンペティションに提出

---

# コメント

> ## Marília Prata
> 
> Llama3とのRiggingの紹介をありがとう、Nick (Nicklanders)。
> 
> 

---